In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')  
from scraper.session import login, BASE_URL
from scraper.fetch import get_court_links_and_dates
# from scraper.parser import parse_cases
from scraper.court_scraper import CourtScraper

from db.db_methods import get_connection, get_court_id_by_city, insert_court_case
from db.models import CourtCase



# getting court links doesnt need session as no log in... but it does want you to log in to view the deetts..
links_and_dates = get_court_links_and_dates()


getting court links


In [3]:

# logging in
session = login()      





probs logged in


In [4]:
# get db connection
get_connection()


<connection object at 0x72a6f9d7efc0; dsn: 'user=jonah password=xxx dbname=courtdb_dev host=localhost port=65432', closed: 0>

In [5]:

for i, (link, date) in enumerate (links_and_dates):
    if i < 1:
        continue
    courtScraper = CourtScraper(session, BASE_URL + link)
    if not courtScraper.load_case_page():
        print(f"struggle to find 'new tab url' for {link}")
        continue
    courtScraper.get_case_list_soup()
    name_for_debug = courtScraper.extract_city_and_court_name()
    court_cases = courtScraper.rows_to_objects(date)
    # print(f"court cases?:{court_cases}")
    if not court_cases:
        print(f"failure to get court cases for: {name_for_debug}")
        continue
    for case in court_cases: # iterate through scraped court cases and add to db
        court_id = get_court_id_by_city(case.city)

        if not court_id and case.city:
            print(f"no court id for {case.city}")
            continue 
        insert_court_case(case, court_id)
    print(f"{name_for_debug} has {len(court_cases)} court cases")


In  the  County Court and the Family     Court at Aldershot & Farnham
Aldershot: number of rows of messy texts containing regex pattern (pre-cases): 21
AA392238 / K07ZA242 UNIVERSITY OF WARWICK V MULLAN
['AA392238', '/', 'K07ZA242', 'UNIVERSITY', 'OF', 'WARWICK', 'V', 'MULLAN']
M00AF047 CHAMBERS V GARDENAL & PEREIRA
['M00AF047', 'CHAMBERS', 'V', 'GARDENAL', '&', 'PEREIRA']
M00AF045 HOBBS & HOBBS V HOGG
['M00AF045', 'HOBBS', '&', 'HOBBS', 'V', 'HOGG']
071DC241 RAZAIE V O'BRIEN
['071DC241', 'RAZAIE', 'V', "O'BRIEN"]
L00AF269 MINISTRY OF DEFENCE v TYLER
['L00AF269', 'MINISTRY', 'OF', 'DEFENCE', 'v', 'TYLER']
M00AF058 STONE V GOULD
['M00AF058', 'STONE', 'V', 'GOULD']
M00AF065 ACCENT HOUSING LTD V BROUGHTON
['M00AF065', 'ACCENT', 'HOUSING', 'LTD', 'V', 'BROUGHTON']
M00AF050 PARTNERSHIP HOME LOANS LTD V THE ESTATE OF K MOSS C/O     PERSONAL REP SARAH HENDERSON
['M00AF050', 'PARTNERSHIP', 'HOME', 'LOANS', 'LTD', 'V', 'THE', 'ESTATE', 'OF', 'K', 'MOSS', 'C/O', '', '', '', '', 'PERSONAL', 'REP'